<a href="https://colab.research.google.com/github/Kirutkar/airesidency/blob/main/stockmarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai==0.28 requests gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.81.0
    Uninstalling openai-1.81.0:
      Successfully uninstalled openai-1.81.0


In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [4]:
import gradio as gr
import openai
import requests
from datetime import datetime

# 🔐 API Key
STOCK_API_KEY = 'XHHWRJYH4VPCGPG1'
openai.api_key = os.getenv("OPENAI_API_KEY")  # Set this in Hugging Face or Colab secrets

# 🔎 Get stock symbol from Alpha Vantage
def get_stock_symbol(company_name):
    url = 'https://www.alphavantage.co/query'
    params = {'function': 'SYMBOL_SEARCH', 'keywords': company_name, 'apikey': STOCK_API_KEY}
    response = requests.get(url, params=params).json()

    try:
        return response['bestMatches'][0]['1. symbol']
    except (KeyError, IndexError):
        return None

# 💰 Get stock quote
def get_stock_quote(symbol):
    url = 'https://www.alphavantage.co/query'
    params = {'function': 'GLOBAL_QUOTE', 'symbol': symbol, 'apikey': STOCK_API_KEY}
    response = requests.get(url, params=params).json()

    try:
        quote = response['Global Quote']
        price = quote['05. price']
        change = quote['09. change']
        percent = quote['10. change percent']
        time = datetime.now().strftime("%A, %B %d – %I:%M %p")

        return f"""
📈 **{symbol.upper()} Price Overview**
- 💵 Current Price: ${price}
- 📉 Change: {change} ({percent})
- 🕒 As of: {time}
""".strip()
    except KeyError:
        return "⚠️ Could not fetch live stock data. Try again later."

# 🤖 Ask GPT
def ask_gpt(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a financial assistant. Provide stock analysis and company info."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.3
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"⚠️ GPT Error: {e}"

# 🔄 Combined logic
def stock_info(company_name, user_question):
    symbol = get_stock_symbol(company_name)
    if not symbol:
        return "❌ Could not find stock symbol for this company.", ""

    stock_price = get_stock_quote(symbol)
    question = user_question.strip() or f"Give an overview of {company_name} stock. Mention its long-term potential and past milestones."
    gpt_response = ask_gpt(question)

    return stock_price, gpt_response

# 🎨 Gradio UI
with gr.Blocks(title="SmartStock Insight – LLM & Market Data") as demo:
    gr.Markdown("### 📊 SmartStock Insight – Powered by GPT + Alpha Vantage")
    gr.Markdown("Enter a company name to get live stock data and ask AI anything about it.")

    company = gr.Textbox(label="🔍 Company Name", placeholder="e.g., Apple, Tesla, Infosys")
    user_q = gr.Textbox(label="🤔 Ask a custom question (optional)", placeholder="e.g., Is it good for long-term?")
    price_box = gr.Textbox(label="💵 Stock Price", lines=4)
    ai_box = gr.Textbox(label="🧠 GPT Insights", lines=10)

    btn = gr.Button("🔎 Analyze Stock")
    btn.click(fn=stock_info, inputs=[company, user_q], outputs=[price_box, ai_box])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd3247d557bef1d471.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
